In [7]:
import csv
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
import matplotlib.pyplot as plt
import math
import antropy as ant
from sklearn.ensemble import RandomForestClassifier

In [8]:
states = ["active", "meditate", "neutral"]
n_samples_per_state = 8

labels = {
    1: "Female",
    2: "Male",
    3: "Male",
    4: "Male",
    5: "Female",
    6: "Female",
    7: "Female",
    8: "Male",
    9: "Male",
    10: "Male",
    11: "Female",
    12: "Male",
    13: "Male",
    14: "Male",
}

class Sample:
    def __init__(self):
        self.data = {
            'RawEEG': [],
            'Alpha': [],
            'Low Beta': [],
            'High Beta': [],
            'Gamma': [],
            'Theta': [],
            'Delta': [],
            'Meditation': [],
            'Attention': []
        }

    def recordDataPoint(self, RawEEG, Attention, Meditation, Alpha, Delta, Theta, LowBeta, HighBeta, Gamma):
        self.data['RawEEG'].append(float(RawEEG))
        self.data['Attention'].append(float(Attention))
        self.data['Meditation'].append(float(Meditation))
        self.data['Alpha'].append(float(Alpha))
        self.data['Delta'].append(float(Delta))
        self.data['Theta'].append(float(Theta))
        self.data['Low Beta'].append(float(LowBeta))
        self.data['High Beta'].append(float(HighBeta))
        self.data['Gamma'].append(float(Gamma))

    '''
    Record a line of data from the CSV output, which takes form RawEEG, Alpha, Delta, Gamma, Low Beta, High Beta, Theta, Attention, Meditation

    '''
    def recordDataLine(self, line):
        self.recordDataPoint(line[0], line[7], line[8], line[1], line[2], line[6], line[4], line[5], line[3])
    
    def getEEG(self):
        return self.data['RawEEG']
    
    def getAttention(self):
        return self.data["Attention"]
    
    def getMeditation(self):
        return self.data["Meditation"]
    
    def getAlpha(self):
        return self.data["Alpha"]
    
    def getDelta(self):
        return self.data["Delta"]
    
    def getTheta(self):
        return self.data["Theta"]
    
    def getLowBeta(self):
        return self.data["Low Beta"]
    
    def getHighBeta(self):
        return self.data["High Beta"]
    
    def getGamma(self):
        return self.data["Gamma"]

    def get(self, key):
        return self.data[key]

    '''
    Filter out all outliers, as defined by being outside 3*std from the mean, and replace with mean of the samples around them
    '''
    def filter_outliers(self):
        for key in self.data:
            data = self.data[key]
            
            filtered = []
            for i, x in enumerate(data):
                if abs(x - np.mean(data)) < 3 * np.std(data):
                    filtered.append(x)
                else:
                    filtered.append(np.mean(data[max(0, i-2):i] + data[i+1:min(len(data), i+2)]))
            self.data[key] = filtered

            # self.data[key] = [x for x in self.data[key] if abs(x - np.mean(self.data[key])) < 3 * np.std(self.data[key])]


In [9]:
data = []
dataLabels = []

def transcribeFileToSample(personN: int, sampleN: int, state: str):
    sample_data = Sample()

    with open(f"data/all_data/{state}_{personN}_{sampleN}.csv") as f:
        reader = csv.reader(f)

        header = next(reader)
        
        for row in reader:
            sample_data.recordDataLine(row)

        sample_data.filter_outliers()

        data.append(sample_data)
        dataLabels.append(labels[personN])

for person in labels:
    for state in states:
        for i in range(n_samples_per_state):
            transcribeFileToSample(person, i + 1, state)

In [10]:
dataExtracted = []

def safety_check(x):
    if math.isnan(x): return 0
    if math.isinf(x): return 99999999999
    return x

for point in data:
    extractedPoint = []

    extractedPoint.append(np.std(point.getGamma()))
    extractedPoint.append(np.mean(point.getTheta()))
    extractedPoint.append(np.mean(point.getGamma()))
    extractedPoint.append(np.mean(point.getHighBeta()))
    extractedPoint.append(np.std(point.getHighBeta()))
    extractedPoint.append(np.std(point.getTheta()))
    extractedPoint.append(np.mean(point.getLowBeta()))
    extractedPoint.append(np.std(point.getDelta()))
    extractedPoint.append(np.mean(point.getAlpha()))
    extractedPoint.append(np.mean(point.getDelta()))
    extractedPoint.append(np.std(point.getLowBeta()))
    extractedPoint.append(safety_check(ant.sample_entropy(point.getDelta())))

    dataExtracted.append(extractedPoint)

In [11]:
from sklearn.model_selection import KFold

cvclf = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=20, n_estimators=2000))
cv = KFold(n_splits=10, shuffle=True, random_state=0)
scores = cross_val_score(cvclf, dataExtracted, dataLabels, cv=cv)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.84 accuracy with a standard deviation of 0.09
